In [207]:
# Helper functions
import os
import csv

PATH = os.getcwd()
def read_file_from_csv(filename):
    # source = '%s%s/%s' % (PATH, pathname, filename)
    df = None
    try:
        df = pd.read_csv(filename, parse_dates=True, infer_datetime_format=True, na_filter=True, quoting=csv.QUOTE_ALL, thousands=',')
        print("read",len(df.index)," rows from", filename)
    except:
        print("error opening the file %s" % filename)
    finally:
        return df



In [206]:
import requests

clientid = 'test'
N = 5
url = 'https://zgvbsyp62d.execute-api.us-east-2.amazonaws.com/dev/matches_broker-dev?clientid={}&N={}'.format(clientid, N)

r = requests.post(url)
if r.status_code == 200:
    print('API call successful. Generating files to match')
    data = r.text
    data = json.loads(data)
    files_to_match = data['matches']

API call successful. Generating files to match


In [208]:
# Get names
import pandas as pd
matches = read_file_from_csv("files/matches.csv")

matches["source_UPC"] = matches["source_UPC"].fillna('')
matches["source_UPC"] = matches["source_UPC"].astype(str) 
matches['source_UPC'] = matches.source_UPC.str.strip()

matches["dest_UPC"] = matches["dest_UPC"].fillna('')
matches["dest_UPC"] = matches["dest_UPC"].astype(str) 
matches['dest_UPC'] = matches.dest_UPC.str.strip()

# Get product names where source UPC/dest UPC is null
missing_source = matches[(matches['source_UPC'] == '') & (matches['source_supplier'] != 'upcitemdb-offer') & (matches['source_supplier'] != 'upcitemdb')]
missing_source_list = missing_source['source_name'].to_list()

missing_dest= matches[(matches['dest_UPC'] == '')  & (matches['dest_supplier'] != 'upcitemdb-offer') & (matches['dest_supplier'] != 'upcitemdb')]
missing_dest_list = missing_dest['dest_name'].to_list()

/var/folders/rj/rrq458bj1mb9fpw62rmd9yb80000gn/T/ipykernel_3983/2428096568.py:3: DtypeWarning: Columns (62) have mixed types.Specify dtype option on import or set low_memory=False.
  matches = read_file_from_csv("files/matches.csv")


read 75313  rows from files/matches.csv


In [209]:
# Combine the 2 and dedupe on names
missing_upcs = missing_source_list + missing_dest_list
missing_upcs = [x.strip() for x in missing_upcs]
print(len(missing_upcs))
missing_upcs = list(set(missing_upcs))
print(len(missing_upcs))


23736
10071


In [210]:
# Filter out names. Pick only those which are not lookedup yet
f = open('files/lookedup')
lookedup_names = f.readlines()
lookedup_names = [l.strip() for l in lookedup_names]
lookedup_names = {l:0 for l in lookedup_names}

missing_upcs = [m for m in missing_upcs if m not in lookedup_names]
len(missing_upcs)

0

In [168]:
# Transformations
og_lookup_names = [n for n in missing_upcs]
tf_lookup_names = [n for n in missing_upcs]
tf_lookup_names = [l.lower() for l in tf_lookup_names]

# Remove oz 
tf_lookup_names = [l.replace('oz.', '') for l in tf_lookup_names]
#tf_lookup_names[:10]

og_lookup_names[:5]

['Fashnpoint White One Sixth Fold Ultra Ply Paper Guest Towel',
 '- Turnbury® Insulated Pedestal Based Bowl 5 oz (48/cs) - Onyx',
 'MICRO ITALIAN BASIL  4oz/EA',
 'MCCORMICK CULINARY POPPY SEED',
 '100 Percentage PCW Paper Compostable Hot Cup Sleeve - 8 Oz.']

In [176]:
f = open('files/out')
items_dic = json.load(f)
len(items_dic)

#items_dic = {}

2000

In [180]:
import requests
import time

headers = {'user_key' : 'f112765d522453bdb002e9b928f7ab1d'}
# 'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:77.0) Gecko/20190101 Firefox/77.0'}
for n, for_url in zip(og_lookup_names[0:], tf_lookup_names[0:]):
    if n not in items_dic or items_dic[n] == 429:
        try:
            url = 'https://api.upcitemdb.com/prod/v1/search?s={}&match_mode=1&type=product'.format(for_url)
            x = requests.get(url, headers=headers, timeout=15)

            data = x.json()
            list_items = []
            for i in data['items']:
                list_items.append(i)
            items_dic[n] = list_items
            print('Found', len(list_items), 'items for', for_url)
        except:
            print('No match found for:', for_url, 'Error code:', x.status_code)
            items_dic[n] = x.status_code
        time.sleep(5.1)

Found 10 items for toilet bowl mop


In [181]:
print(len(items_dic))
# Export result
import json
with open('files/out', 'w') as fout:
    json.dump(items_dic, fout)

2247


In [182]:
# Export names that are already searched
with open('files/lookedup', 'a') as fout:
    fout.write("\n".join(og_lookup_names))
    fout.write("\n")

In [183]:
# Export names with 404 error
error_404 = []
for k,v in items_dic.items():
    if type(v) is not list:
        error_404.append(k)

f = open('files/error_404')
_tmp = f.readlines()
_tmp = [l.strip() for l in _tmp]
_tmp = {l:0 for l in _tmp}

error_404 = [e for e in error_404 if e not in _tmp]

with open('files/error_404', 'a') as fout:
    fout.write("\n".join(error_404))
    fout.write("\n")

In [184]:
# Explore available columns in data
from datetime import datetime

columns = ['lookup_name', 'product_url', 'price', 'is_offer']

for k,v in items_dic.items():
    if type(v) is list:
        for _ in v:
            for _k, _v in _.items():
                if _k not in columns and _k != 'offers':
                    columns.append(_k)

columns.append('updated_t')
# Create item list and equivalent csv
results = []
for k,v in items_dic.items():
    if type(v) is list:
        for _ in v:
            default_dic = {}
            for c in columns:
                default_dic[c] = ''
            default_dic['updated_t'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
            default_dic['lookup_name'] = k
            for _k, _v in _.items():
                if _k != 'offers':
                    default_dic[_k] = _v
            results.append(dict(default_dic))
            for o in _['offers']:
                default_dic['product_url'] = o['link']
                default_dic['title'] = o['title']
                default_dic['price'] = o['price']
                default_dic['is_offer'] = 1
                ts = int(o['updated_t'])
                # if you encounter a "year is out of range" error the timestamp
                # may be in milliseconds, try `ts /= 1000` in that case
                default_dic['updated_t'] = datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
                results.append(dict(default_dic))
        

enriched = pd.DataFrame(results)
enriched.to_csv('files/enriched.csv', index=False)

In [ ]:
c = 0
for k,v in items_dic.items():
    if v == 404:
        c+=1
c
len(og_lookup_names)

In [ ]:
for k,v in items_dic.items():
    if type(v) is list:
        print(v)
        break

In [ ]:
# Random
# x = re.findall(r"[ \t]*(?:\d|[^\w\s])+[ \t]*", x) # Extract the numeric parts

import re
a = 'Gojo, GOJ965212CT, Instant Hand Sanitizer, 12 / Carton, Clear'
#a = 'AJM Packaging Corporation White Paper Plates, 9" Diameter, 100 Count'
#a = 'Keebler Club Crackers Original .25oz'
a = 'Package Size: 12x 6oz'
look_ahead = "gallon|each|oz|ml|lb|gram|CT|oz|fl oz|\""

ss = '(\d+(?:\.\d*)?\\s?(?=({})(?:\\s|$|,)))'.format(look_ahead)

ss = '((?:^|\s)\d*\.?\d+?\\s?(?=({})(?:\\s|$|,)))'.format(look_ahead)



print(re.findall(ss, a))


In [ ]:
def foo(x):
        x = x.lower()
        if 'pack of' in x:
            x = x.split('pack of')
            print(re.findall('\d*\.?\d+?', x[1]))
            try:
                return int(re.findall('\d*\.?\d+?', x[1])[0])
            except:
                ''
        elif 'pack' in x:
            x = x.split('pack')
            try:
                return int(re.findall('\d*\.?\d+?', x[0])[0])
            except:
                ''
        return ''
a='8 Ounce (Pack of 1)'
foo(a)
#re.findall('\d*\.?\d+?', 'asd (23.3')

In [ ]:
x= '''
Package Size: 
							12ct/CS
'''

x = x.replace('#', '')
x = x.split('/')
if 'ct' in x[0]:
    x = x[0]

x.split('ct')[0].strip().split(':')[-1].strip()



In [ ]:
re.findall(r'\b\d+\b', 'Pack: 1Case')